In [ ]:
!pip install dash==1.4.1
!pip install dash-daq==0.2.1

In [61]:
%%sh
pip install -q dash_core_components
pip install -q dash_html_components
pip install -q dash_table

In [62]:
%%sh
# get ngrok
curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.1M  100 13.1M    0     0  25.1M      0 --:--:-- --:--:-- --:--:-- 25.1M
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [79]:
%%writefile my_app3.py
import os
import pathlib
import re
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from dash.dependencies import Input, Output, State
import cufflinks as cf
import numpy as np
from dash.exceptions import PreventUpdate
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from scipy.optimize import curve_fit

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__,external_stylesheets=external_stylesheets)
server = app.server

def prepare_daily_report():

  df = pd.read_csv('./WHO_data_noncom.csv')
  
  return df

df1 = prepare_daily_report()

def prepare_daily():

  df = pd.read_csv('./data_10000_noncom_smoking_BMI_Alc_GDP.csv') 
  df = df.drop(columns='Unnamed: 0') 
  return df

def life_exp():

  df = pd.read_csv('./life_exp.csv') 

  return df

all_d = prepare_daily()

low = all_d[all_d['low'] == True]
low = low.drop(columns = ['Unnamed: 0.1','Unnamed: 0.1.1','Time','CDNONCom','CDComm', 'CDInj', 'Region', 'Country Name', 'Country Code','Population', 'low', 'upper_low', 'under_high', 'high' ])
low = low.rename(columns={"Smoking prev":"Smoking prevalence", "Consumption per 10000 in l":"Alcohol consumption",
                          "BMI":"Overweight", "GDP for health per capita":"Domestic health expenditure of the state"})
low = low.corr()

fig = make_subplots(rows=2, cols=2, specs=[[{"type": "heatmap"}, {"type": "heatmap"}],
           [{"type": "heatmap"}, {"type": "heatmap"}]], shared_yaxes=True, shared_xaxes=True,
                    subplot_titles=("low-income<br>health economies", "lower middle-income<br>health economies", 
                                    "upper middle-income<br>health economies", 
                                    "high-income<br>health economies"))

col = []
for column in low.columns:
  col.append(column)

z = []
for i, j in low.iteritems():
  z.append(j.values)
  
fig.add_trace(go.Heatmap(
                   z= z,
                   x=col[:4],
                   y=col,
                   type = 'heatmap',
                   colorscale = 'Blues',
                   hoverongaps = False,
                   coloraxis="coloraxis",
                   #colorbar={"thickness": 10,"len": 0.7,"x": 1.3,"y": 0.5,}
                   ), row=1, col=1)

upper = all_d[all_d['upper_low'] == True]
upper = upper.drop(columns = ['Unnamed: 0.1','Unnamed: 0.1.1','Time','CDNONCom','CDComm', 'CDInj', 'Region', 'Country Name', 'Country Code','Population', 'low', 'upper_low', 'under_high', 'high' ])
upper = upper.rename(columns={"Smoking prev":"Smoking prevalence", "Consumption per 10000 in l":"Alcohol consumption",
                          "BMI":"Overweight", "GDP for health per capita":"Domestic health expenditure of the state"})
upper = upper.corr()

colu = []
for column in upper.columns:
  colu.append(column)

zu = []
for i, j in upper.iteritems():
  zu.append(j.values)
  
fig.add_trace(go.Heatmap(
                   z= zu,
                   x=colu[:4],
                   y=colu,
                   type = 'heatmap',
                   colorscale = 'Blues',
                   hoverongaps = False,
                   coloraxis="coloraxis",
                   #colorbar={"thickness": 10,"len": 0.7,"x": 1.3,"y": 0.5,}
                                    ), row=1, col=2)


under_high = all_d[all_d['under_high'] == True]
under_high = under_high.drop(columns = ['Unnamed: 0.1','Unnamed: 0.1.1','Time','CDNONCom','CDComm', 'CDInj', 'Region', 'Country Name', 'Country Code','Population', 'low', 'upper_low', 'under_high', 'high' ])
under_high = under_high.rename(columns={"Smoking prev":"Smoking prevalence", "Consumption per 10000 in l":"Alcohol consumption",
                          "BMI":"Overweight", "GDP for health per capita":"Domestic health expenditure of the state"})
under_high = under_high.corr()

coluh = []
for column in under_high.columns:
  coluh.append(column)

zuh = []
for i, j in under_high.iteritems():
  zuh.append(j.values)
  
fig.add_trace(go.Heatmap(
                   z= zuh,
                   x=coluh[:4],
                   y=coluh,
                   type = 'heatmap',
                   colorscale = 'Blues',
                   hoverongaps = False,
                   coloraxis="coloraxis",
                   #colorbar={"thickness": 10,"len": 0.7,"x": 1.3,"y": 0.5,}
                                    ), row=2, col=1)

high = all_d[all_d['high'] == True]
high = high.drop(columns = ['Unnamed: 0.1','Unnamed: 0.1.1','Time','CDNONCom','CDComm', 'CDInj', 'Region', 'Country Name', 'Country Code','Population', 'low', 'upper_low', 'under_high', 'high' ])
high = high.rename(columns={"Smoking prev":"Smoking prevalence", "Consumption per 10000 in l":"Alcohol consumption",
                          "BMI":"Overweight", "GDP for health per capita":"Domestic health expenditure of the state"})
high = high.corr()

colh = []
for column in high.columns:
  colh.append(column)

zh = []
for i, j in high.iteritems():
  zh.append(j.values)
  
fig.add_trace(go.Heatmap(
                   z= zh,
                   x=colh[:4],
                   y=colh,
                   type = 'heatmap',
                   colorscale = 'Blues',
                   hoverongaps = False,
                   coloraxis="coloraxis",
                   #colorbar={"thickness": 10,"len": 0.7,"x": 1.3,"y": 0.5,}
                                    ), row=2, col=2)

fig.update_layout(title = dict(
          x=0.5,
          y=0.95,
           text="Correlations between <b>non-communicable diseases</b> and<br>their <b>risk factors</b> for countries with:",
            font=dict(family="Open Sans, sans-serif", size=15),),
                  coloraxis=dict(colorscale='Blues'), height = 750, width = 600)
for i in fig['layout']['annotations']:
    i['font'] = dict(size=10, color='#696969')

app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Future Challenges For Global Health', children=[
                                                                       
      html.Div(id ='slider row',
               className = 'row',
               children=[            
			html.Div([
				html.P('Drag the slider to change the year:'),
			  ]),

			html.Div([
				dcc.Slider(
          id="years-slider",
          min=df1['Time'].min(),
					max=df1['Time'].max(),
					value=df1['Time'].min(),
          step = None,
          marks={str(year): str(year)
          for year in df1['Time'].unique()}
          ),
			], style={'width':550, 'margin':10}),
      ], style={'margin': 20}),
                                                                       
      html.Div(
          id = 'first row',
          className='row',
          children=[                            
	html.Div([
    dcc.Graph(id="life-expect"),
	], className='six columns', style={'margin':0}),

	html.Div([
		dcc.Graph(
      id="deaths-data", ),
	], className='six columns', style={'margin':0}),],),

    html.Div(
    id="second-row",
    className="row",
    children=[
      html.Div([
        html.P('Select cause of death:', style={'display': 'inline-block'}),
		    dcc.Dropdown(
          options=[{'label': "Cause of death by non-communicable diseases ", 'value': 'CDNONCom'},
              {'label': "Cause of death by communicable diseases ", 'value': 'CDComm'},
              {'label': "Cause of death by injury ", 'value': 'CDInj'}],
              value="CDNONCom",
              id="chart-dropdown",
              ),
            dcc.Graph(id="county-choropleth"),
      ],className='six columns', style={'margin':0}),
      
      html.Div([
        dcc.Graph(
          id="selected-data",
          figure=dict(
          data=[dict(x= list(year for year in df1['Time'].unique()), y=0)],
          layout=dict(
          autofill=True,
          margin=dict(t=75, r=50, b=100, l=50),
          ),
        ),
      ),            
      ],className='six columns', style={'margin':0}),
    ],),


      
      ]),

      dcc.Tab(label='Risk Factors For Non-Communicable Diseases', children=[
                                         
       html.Div(
          id = 'first row1',
          className='row',
          children=[                            
	html.Div([
    html.P('Select a health risk factor:', style={'display': 'inline-block'}),
		dcc.Dropdown(
      options=[{'label': "Smoking prevalence ", 'value': 'Smoking prev'},
              {'label': "Prevalence of overweight ", 'value': 'BMI'},
              {'label': "Prevalence of alcohol consumption in liter ", 'value': 'Consumption per 10000 in l'}],
              value="Smoking prev",
              id="chart-dropdown1",
              ),
		
	], className='three columns', style={'margin':20}),

	html.Div([
		html.P('Select an economical factor:', style={'display': 'inline-block'}),
		dcc.Dropdown(
      options=[{'label': "Low-income health economies ", 'value': 'low'},
               {'label': "Lower middle-income health economies ", 'value': 'upper_low'},
               {'label': "Upper middle-income health economies ", 'value': 'under_high'},
               {'label': "High-income health economies ", 'value': 'high'}
               ],
              value="low",
              id="chart-dropdown2",
              ),
	], className='three columns', style={'margin':20}),

  html.Div([
		html.P('Select a non-communicable disease:', style={'display': 'inline-block'}),
		dcc.Dropdown(
      options=[{'label': "Malignant neoplasms ", 'value': 'Malignant neoplasms'},
              {'label': "Diabetes mellitus ", 'value': 'Diabetes mellitus'},
              {'label': "Chronic obstructive pulmonary disease ", 'value': 'Chronic obstructive pulmonary disease'},
              {'label': 'Cardiovascular diseases ', 'value': 'Cardiovascular diseases'}],
              value="Cardiovascular diseases",
              id="chart-dropdown3",
              ),
	], className='three columns', style={'margin':20}),
  ],),  


  html.Div(
    id="second-row1",
    className="row",
    children=[
      html.Div([
        html.Div([            
			    html.Div([
				    html.P('Drag the slider to change the year:'),
			    ]),
			  html.Div([
				dcc.Slider(
          id="years-slider1",
          min=all_d['Time'].min(),
					max=all_d['Time'].max(),
					value=all_d['Time'].min(),
          step = None,
          marks={str(year): str(year)
          for year in all_d['Time'].unique()}
          ),
			  ], style={'width':550, 'margin':10}),
        ], style={'margin': 20} ),
            dcc.Graph(id="choropleth"),
      ],className='six columns', style={'margin':0}),
      
      html.Div([
        html.Div([ 
          html.Div([
				    html.P('Select countries:'),
			    ]),
			  html.Div([
        dcc.Dropdown(
          id="operator-select",
          options=[
            {"label": i, "value": i}
            for i in all_d["Country Name"].unique().tolist()
            ],
          multi=True,
          value=[
            all_d["Country Name"].unique().tolist()[0],
            all_d["Country Name"].unique().tolist()[1],
            ],
        ),
        ], style={'width':550, 'margin':10}),
        ], style={'margin': 20} ),
        dcc.Graph(
          id="histogram",
      ),            
      ],className='six columns', style={'margin':0}),
    ],),  

    html.Div(
          id = 'third row1',
          className='row',
          children=[                            
	html.Div([
    html.P('Select an item for the x-axis:', style={'display': 'inline-block'}),
		dcc.Dropdown(
      options=[{'label': "Smoking prevalence ", 'value': 'Smoking prev'},
              {'label': "Prevalence of overweight ", 'value': 'BMI'},
              {'label': "Prevalence of alcohol consumption in liter ", 'value': 'Consumption per 10000 in l'},
               {'label': 'Domestic general government health expenditure in $ ', 'value': 'GDP for health per capita'},
              {'label': "Malignant neoplasms ", 'value': 'Malignant neoplasms'},
              {'label': "Diabetes mellitus ", 'value': 'Diabetes mellitus'},
              {'label': "Chronic obstructive pulmonary disease ", 'value': 'Chronic obstructive pulmonary disease'},
              {'label': 'Cardiovascular diseases ', 'value': 'Cardiovascular diseases'}],
              value='Cardiovascular diseases',
              id="chart-dropdown5",
              ),
		
	], className='three columns', style={'margin':20}),

	html.Div([
		html.P('Select an item for the y-axis:', style={'display': 'inline-block'}),
		dcc.Dropdown(
      options=[{'label': "Smoking prevalence ", 'value': 'Smoking prev'},
              {'label': "Prevalence of overweight ", 'value': 'BMI'},
              {'label': "Prevalence of alcohol consumption in liter ", 'value': 'Consumption per 10000 in l'},
               {'label': 'Domestic general government health expenditure in $ ', 'value': 'GDP for health per capita'},
               {'label': "Malignant neoplasms ", 'value': 'Malignant neoplasms'},
              {'label': "Diabetes mellitus ", 'value': 'Diabetes mellitus'},
              {'label': "Chronic obstructive pulmonary disease ", 'value': 'Chronic obstructive pulmonary disease'},
              {'label': 'Cardiovascular diseases ', 'value': 'Cardiovascular diseases'}],
              value="Smoking prev",
              id="chart-dropdown4",
              ),
	], className='three columns', style={'margin':20}),
  ],),   

    html.Div(
    id="fourth-row1",
    className="row",
    children=[
      html.Div([
        dcc.Graph(
          id="classified",
      ),            
      ],className='six columns', style={'margin':0}),

      html.Div([
        dcc.Graph(id="heatmap",
          figure = fig
                ),
      ],className='six columns', style={'margin':0}),
      
      
    ],),                            
      
      ]),
    ])
])

#############################@app.callback's for the first tab#################################
@app.callback(
    Output("county-choropleth", "figure"),
    [Input("years-slider", "value"),
    Input("chart-dropdown", "value"),
    Input("chart-dropdown", "options")],
)

def display_map(year, dropdown, dropdown_o):
  dff = prepare_daily_report()
  dff1 = dff[dff['Time'] == year]
  dff1['hover_text'] = dff1["Country Name_y"].apply(str) + ": " + dff1[dropdown].apply(str) 

  for i in dropdown_o:
    if (i['value'] == dropdown):
      text_drop = i['label']
  
  trace = go.Choropleth(locations=dff1['Country Code_x'],
                          z=dff1[dropdown],
                          text=dff1['hover_text'],
                          hoverinfo="text",
                          marker_line_color='white',
                          autocolorscale=False,
                          reversescale=True,
                          colorscale="Magma",marker={'line': {'color': 'rgb(180,180,180)','width': 0.5}},
                          colorbar={"thickness": 10,"len": 0.7,"x": 1.3,"y": 0.5,
                                    } )   
  title = ("3D-Choropleth map")

  layout = dict(
        title=dict(
            text='Prevalence of: <b>' + text_drop + '</b>in <b>' + str(year) + '</b>,<br> in %',
            font=dict(family="Open Sans, sans-serif", size=18),
        ),
        margin=dict(l= 60, b = 50, t = 80, r = 0, pad=2),
        automargin=False,
        clickmode="event+select",
        geo=go.layout.Geo(
            showland = True,
            showcountries = True,
            showframe = True,
            showcoastlines = True,
            landcolor="white",
            oceancolor="DarkBlue",
            showocean=True,
            lakecolor="LightBlue",
            projection=go.layout.geo.Projection(type="orthographic")
        ),
    )

  return {"data": [trace],
            "layout": layout}


@app.callback(
    Output('selected-data', 'figure'),
    [Input('county-choropleth', 'clickData')],
    [State('county-choropleth', 'figure')]
)

def display_selected_data(hoverData, figure):
  df = prepare_daily_report()

  title='Click on the globe to select a country'

  if hoverData is None:
    return dict(
		data = [dict(x= list(year for year in df['Time'].unique()), y=0)],
		layout = dict(
      title = dict(
				text = title,
        font=dict(family="Open Sans, sans-serif", size=20)
			) ))
  country_name = hoverData['points'][0]['text'].split(':')[0]
  dff = df[df['Country Name_y'] == country_name]    

  trace1 = go.Scatter(
            x=dff['Time'],
            y=dff['CDNONCom'],
            text='Country:' + country_name ,
            name = 'Cause of death by<br>non-communicable diseases in %',
            mode='lines+markers',
            line_shape='spline',
            marker={
                'size': 15,
                'opacity': 0.5,
                'color': 'rgba(38, 24, 74, 0.8)',
                'line': {'width': 0.5, 'color': 'rgba(38, 24, 74, 0.8)'}
            }
        )

  trace2 = go.Scatter(
            x=dff['Time'],
            y=dff['CDComm'],
            text='Country:' + country_name ,
            name = 'Cause of death by<br>communicable diseases in %',
            mode='lines+markers',
            line_shape='spline',
            marker={
                'size': 15,
                'opacity': 0.5,
                'color': 'rgba(122, 120, 168, 0.8)',
                'line': {'width': 0.5, 'color': 'rgba(71, 58, 131, 0.8)'}
            }
        )

  trace3 = go.Scatter(
            x=dff['Time'],
            y=dff['CDInj'],
            text='Country:' + country_name ,
            name = 'Cause of death by<br>injury in %',
            mode='lines+markers',
            line_shape='spline',
            #fill='tozeroy',
            marker={
                'size': 15,
                'opacity': 0.5,
                'color': 'rgba(190, 192, 213, 1)',
                'line': {'width': 0.5, 'color': 'rgba(122, 120, 168, 0.8)'}
            }
        )


  return {
        'data': [trace1,trace2,trace3] ,
        'layout': go.Layout(
            title = dict(
				    text = '<b>' + country_name + '</b>',
            font=dict(family="Open Sans, sans-serif", size=20)),
            xaxis={'title': 'Year'},
            yaxis={'title': 'Values in %'},
            #margin={'l': 60, 'b': 50, 't': 80, 'r': 0},
            hovermode='closest'
        )
    }

@app.callback(
    Output('life-expect', 'figure'),
    [Input("years-slider", "value"),]
)

def display_life_expectancy(v):

  df = life_exp()
  df1 = df[df['Time']==v]

  fig = px.histogram(df1, x=df1["Life expectancy"], color="Region", 
                     marginal="rug", hover_name="Country Name",
                     color_discrete_sequence=px.colors.qualitative.Prism)
                     

  fig.update_layout(
      title = dict(
          x=0.5,
			    text = 'Life expectancy at birth in <b>' + str(v) + '</b>',
            font=dict(family="Open Sans, sans-serif", size=18),),
      width=650, height=600,
      #xaxis_title = str(text_disease),
      #yaxis_title = 'Number of the countries ',
            )
  
  return fig

@app.callback(
    Output('deaths-data', 'figure'),
    [Input("years-slider", "value"),]
)
def display_deaths_histrogramm(v):

  df_region = prepare_daily_report()
  grouped = df_region.groupby(['Region', 'Time'])
  distr = grouped['CDNONCom', 'CDComm', 'CDInj'].agg([np.mean])
  distr = distr.reset_index(level=['Region','Time'])
  distr['CDNONCom', 'mean'] = distr['CDNONCom', 'mean'].round(decimals=2)
  distr['CDComm', 'mean'] = distr['CDComm', 'mean'].round(decimals=2)
  distr['CDInj', 'mean'] = distr['CDInj', 'mean'].round(decimals=2) 

  distr1 = distr[distr['Time'] == v]

  trace1 = go.Bar(
        y=distr1['Region'],
        x=distr1['CDNONCom', 'mean'],
        name='deaths by<br>non-communicable<br>diseases',
        text='Cause of death<br>by non-communicable diseases in %',
        orientation='h',
        marker=dict(
        color='rgba(38, 24, 74, 0.8)',
      ))
  
  trace2 = go.Bar(
        y=distr1['Region'],
        x=distr1['CDComm', 'mean'],
        text = 'Cause of death<br>by communicable diseases and maternal,<br>prenatal and nutrition conditions in%',
        name='deaths by<br>communicable<br>diseases',
        orientation='h',
        marker=dict(
        color='rgba(71, 58, 131, 0.8)',
      ))
  
  trace3= go.Bar(
        y=distr1['Region'],
        x=distr1['CDInj', 'mean'],
        name='deaths by<br>injury',
        text = 'Cause of death<br>by injury in %',
        orientation='h',
        marker=dict(
        color='rgba(122, 120, 168, 0.8)',
      ))


  return {
        'data': [trace1,trace2,trace3] ,
        'layout': go.Layout(
            barmode='stack',
            height = 450,
            width = 600,
            title = dict(
				    text = 'Cause of deaths in <b>' + str(v) + '</b>',
            font=dict(family="Open Sans, sans-serif", size=20)),
            margin=go.layout.Margin(
            l=125),
            hovermode='closest'
        )
    }

#############################@app.callback's for the second tab#########################################


@app.callback(
    Output("choropleth", "figure"),
    [Input("years-slider1", "value"),
    Input("chart-dropdown2", "value"),
    Input("chart-dropdown2", "options"),
    Input("chart-dropdown3", "value"),
     Input("chart-dropdown3", "options")],
)

def display_map(year, gdp, gdp_o, dis, dis_o):
  dff = prepare_daily()
  dff1 = dff[dff['Time'] == year]
  dff2 = dff1[dff1[gdp]==True]
  dff2['hover_text'] = dff2["Country Name"].apply(str) + ": " + dff2[dis].apply(str) 

  for i in gdp_o:
    if (i['value'] == gdp):
      text_gdp = i['label']

  for i in dis_o:
    if (i['value'] == dis):
      text_dis = i['label']
  
  trace = go.Choropleth(locations=dff2['Country Code'],
                          z=dff2[dis],
                          text=dff2['hover_text'],
                          hoverinfo="text",
                          marker_line_color='white',
                          autocolorscale=False,
                          reversescale=True,
                          colorscale="Magma",marker={'line': {'color': 'rgb(180,180,180)','width': 0.5}},
                          colorbar={"thickness": 10,"len": 0.7,"x": 1.3,"y": 0.5,
                                    } )   

  layout = dict(
        title=dict(
            text='Number of deaths attributed to <b>' + text_dis + '</b>per 10,000 people<br>for countries with <b>' + 
            text_gdp + '</b>in <b>'+ str(year) + '</b>',
            font=dict(family="Open Sans, sans-serif", size=14),
        ),
        margin=dict(l= 60, b = 50, t = 80, r = 20, pad=2),
        automargin=False,
        clickmode="event+select",
        geo=go.layout.Geo(
            showland = True,
            showcountries = True,
            showframe = True,
            showcoastlines = True,
            landcolor="white",
            oceancolor="DarkBlue",
            showocean=True,
            lakecolor="LightBlue",
            projection=go.layout.geo.Projection(type="miller")
        ),
    )

  return {"data": [trace],
            "layout": layout}

def exponential_fit(x, a, b, c):
    return a*np.exp(-b*x) + c 

@app.callback(
    Output("histogram", "figure"),
    [Input("operator-select", "value"),
    Input("chart-dropdown1", "value"),
    Input("chart-dropdown1", "options"),
    Input("chart-dropdown3", "value"),
    Input("chart-dropdown3", "options"),],
)

def compare_country(countr, risk, risk_o, diseas, diseas_o):

  df = prepare_daily()
  
  for i in risk_o:
    if (i['value'] == risk):
      text_risk = i['label']

  for i in diseas_o:
    if (i['value'] == diseas):
      text_diseas = i['label']

  trace_colors = ['#A52A2A', '#0000FF', '#2F4F4F', '#C71585', '#808000', '#FA8072', '#2E8B57', '#FFE4B5', '#4B0082']

  trace = []
  for i in countr:
    dff = df[df["Country Name"]==i]
    i = go.Bar(
        y=dff[risk],
        x=dff['Time'],
        legendgroup="group",
        name=text_risk +'<br>' + i,
        text= text_risk,
        marker=dict(
        
      ))
    trace.append(i)

  
  for c,j in zip(trace_colors,countr):
    dff1 = df[df["Country Name"]==j]
    color = c 
    j = go.Scatter(
        y=dff1[diseas],
        x=dff1['Time'],
        legendgroup="group1",
        line_shape='spline',
        line = dict(color = color),
        name=text_diseas +'<br>'+j,
        text= text_diseas,
        )
    trace.append(j)

  for co,k in zip(trace_colors,countr):
    dff2 = df[df["Country Name"]==k]
    fitting_parameters, covariance = curve_fit(exponential_fit, dff2['Time'], dff2[diseas])
    a, b, c = fitting_parameters

    year = []
    next_x = 2017
    for y in dff2['Time']:
      year.append(y)
    year.insert(5,next_x)

    next_y = exponential_fit(next_x, a, b, c)
    y_new = []
    for i in dff2[diseas]:
      y_new.append(i)
    y_new.insert(5,next_y)

  ##########fitting for 2019#####################
    fitting_parameters1, covariance1 = curve_fit(exponential_fit, year, y_new)
    a1, b1, c1 = fitting_parameters1

    year1 = []
    next_xx = 2019
    for y in year:
      year1.append(y)
    year1.insert(6,next_xx)

    next_y1 = exponential_fit(next_xx, a1, b1, c1)
    y_new1 = []
    for i in y_new:
      y_new1.append(i)
    y_new1.insert(6,next_y1)

    color = co 
    k = go.Scatter(
        y=y_new1[4:7],
        x=[2016, 2017, 2019],
        legendgroup="group2",
        name='Trendline for <br>' + text_diseas +'<br>'+k,
        text= text_diseas,
        line_shape='spline',
        line = dict(dash='dash', color = color))
    trace.append(k)
  
  return {
        'data': trace,
        'layout': go.Layout(
            height = 450,
            width = 600,
            yaxis_type="log",
            xaxis_type='category',
            title = dict(
				    text = '<b>' + text_risk + '</b>per 10,000 people vs. <br>Number of deaths attributed to <b>' + text_diseas + '</b><br> per 10,000 people',
            font=dict(family="Open Sans, sans-serif", size=15)),
            margin=go.layout.Margin(
            l=100),
            xaxis={'title': 'Year'},
            yaxis={'title': 'Values'},
            hovermode='closest'
        )
    }

@app.callback(
    Output("classified", "figure"),
    [Input("years-slider1", "value"),
     Input("chart-dropdown4", "value"),
     Input("chart-dropdown4", "options"),
     Input("chart-dropdown5", "value"),
     Input("chart-dropdown5", "options")],
)


def statistical_plot(year, factor, factor_l, disease, disease_l):
  df = prepare_daily()
  df1 = df[df['Time'] == year]

  for i in factor_l:
    if (i['value'] == factor):
      text_factor = i['label']

  for i in disease_l:
    if (i['value'] == disease):
      text_disease = i['label']

  fig = px.scatter(df1, x=df1[disease], y=df1[factor], color="Region",
                   marginal_x="box",
                   #marginal_y="violin",# or violin, rug
                   hover_data=['Country Name'],
                   )
  fig.update_layout(
      title = dict(
          x=0.5,
			    text = 'Correlation betwenn <b>' + text_disease + '</b>per 10,000 people and <br> <b>' + text_factor + 
          '</b>per 10,000 people in <b>'+ str(year) + '</b>',
            font=dict(family="Open Sans, sans-serif", size=14),),
      width=650, height=600,
      xaxis_title = str(text_disease),
      yaxis_title = str(text_factor),
            )
  
  return fig

if __name__ == '__main__':
  app.run_server(host='127.0.0.1',port=8000,debug=True)

Overwriting my_app3.py


In [80]:
### This command runs in the background.
get_ipython().system_raw('./ngrok http 8000 &')

In [81]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://d8c96af2d216.ngrok.io


In [82]:
!python my_app3.py

Running on http://127.0.0.1:8000/
Debugger PIN: 444-280-532
 * Serving Flask app "my_app3" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Running on http://127.0.0.1:8000/
Debugger PIN: 475-351-117
/content/my_app3.py:557: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:808: OptimizeWarning:

Covariance of the parameters could not be estimated

